**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
sudo docker run --rm -v <ruta a mi directorio>:/datos --cap-add SYS_ADMIN --privileged --name jupyterlab-numerical -p 8888:8888 -d palmoreck/jupyterlab_numerical:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_numerical
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

---

Nota generada a partir de [liga](https://www.dropbox.com/s/fyqwiqasqaa3wlt/3.1.1.Multiplicacion_de_matrices_y_estructura_de_datos.pdf?dl=0), [liga2](https://www.dropbox.com/s/l4hq45rj860ql9f/3.1.2.Localidad_y_vectorizacion.Analisis_del_error_en_computos_matriciales_basicos.pdf?dl=0)

# 3.1 El cómputo matricial y el álgebra lineal. Vectorización, BLAS y el uso del caché eficientemente.

El cómputo matricial está construído sobre una jerarquía de operaciones del álgebra lineal:

* Productos punto involucran operaciones escalares de suma y multiplicación (nivel BLAS 1).

* La multiplicación matriz-vector está hecha de productos punto (nivel BLAS 2).

* La multiplicación matriz-matriz utiliza colecciones de productos matriz-vector (nivel BLAS 3).

Las operaciones anteriores se describen en el álgebra lineal con la teoría de espacios vectoriales pero también es posible describirlas en una forma algorítmica. Ambas descripciones se complementan una a la otra.

Manejaremos nombres que en el [Linear Algebra Package: LAPACK](http://www.netlib.org/lapack/explore-html/dir_fa94b7b114d387a7a8beb2e3e22bf78d.html) son utilizados para denotar algunas operaciones con escalares, vectores o matrices. Ver [ Reference-LAPACK / lapack](https://github.com/Reference-LAPACK/lapack) para su github.

## Operación del producto interno estándar o producto punto

Consideramos $x,y \in \mathbb{R}^n$. El producto punto entre $x$ y $y$ es $c = x^Ty = \displaystyle \sum_{i=1}^n x_iy_i$. 

**Ejemplo y algoritmo del producto punto:**

In [21]:
c=0
n=5
x=[-1]*n
y=[1.5]*n

for i in range(n):
    c += x[i]*y[i]

In [22]:
c

-7.5

**Obs:**

* El producto punto de dos $n$-vectores involucran $n$ multiplicaciones y $n$ sumas para un total de $2n$ operaciones. Usamos la notación $\mathcal{O}(\cdot)$ para escribir que el producto punto es $\mathcal{O}(n)$ y se lee "de orden $n$ o proporcional a $n$" para indicar que la **cantidad de trabajo** tiene un comportamiento **lineal** con la dimensión $n$. También tal cantidad de trabajo opera sobre una **cantidad lineal de datos**.

## Operación **saxpy**

Consideramos $\alpha \in \mathbb{R}, x,y \in \mathbb{R}^n$. El nombre lo recibe por *scalar alpha x plus y*. En LAPACK se escribe en forma *update*:

$$y=\alpha x + y \therefore y_i = \alpha x_i + y_i \forall i=1,...,n$$

**Obs:** 

* El símbolo $=$ no se utiliza como igualdad de expresiones sino como en computación para denotar asignación.

* También encontramos en LAPACK `caxpy` o `daxpy` para el caso complejo y para números en doble precisión respectivamente.

* Ésta operación realiza un trabajo de $\mathcal{O}(n)$ sobre una cantidad de datos $\mathcal{O}(n)$.

**Ejemplo y algoritmo de saxpy:**

In [23]:
alpha=2
n=5
x=[-2]*n
y=[0]*n

for i in range(n):
    y[i] += alpha*x[i]

In [24]:
y

[-4, -4, -4, -4, -4]

o en una forma *update*:

In [25]:
alpha=2
n=5
x=[-2]*n
y=[3,4,-1,0,1]

for i in range(n):
    y[i] += alpha*x[i]

In [26]:
y

[-1, 0, -5, -4, -3]

**Comentario:** La operación de producto punto y *saxpy* son algoritmos catalogados como de **nivel BLAS 1** (ver [BLAS: Basic Linear Algebra Subprograms](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms). Éstos algoritmos se caracterizan por involucrar una cantidad de trabajo lineal sobre una cantidad lineal de datos. Ver [level 1](http://www.netlib.org/blas/#_level_1) para más ejemplos.

## Multiplicación matriz-vector

Consideramos $A \in \mathbb{R}^{m \times n}, x \in \mathbb{R}^n, y \in \mathbb{R}^m$. La operación $y = y + Ax$ es una operación *generalizada* saxpy, por ello se denomina **gaxpy** pero en LAPACK podemos encontrar esta operación con nombres como [sgemv](http://www.netlib.org/lapack/explore-html/db/d58/sgemv_8f.html), [dgemv](http://www.netlib.org/lapack/explore-html/dc/da8/dgemv_8f.html), [cgemv](http://www.netlib.org/lapack/explore-html/d4/d8a/cgemv_8f.html) o [zgemv](http://www.netlib.org/lapack/explore-html/db/d40/zgemv_8f.html) para los casos de precisión simple, doble o números complejos respectivamente. Hay diferentes formas de visualizar y escribir el algoritmo de multiplicación matriz-vector. Por ejemplo para una matriz $A$ con entradas:


In [56]:
m=2
n=5
A=[[1.2]*n if i%2==0 else [1]*n for i in range(m)]

In [57]:
A

[[1.2, 1.2, 1.2, 1.2, 1.2], [1, 1, 1, 1, 1]]

In [58]:
A[0][0]

1.2

In [59]:
A[1][n-1]

1

se tiene:

### Algoritmo gaxpy *row oriented*

In [60]:
x=[2]*n
y=[0]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [61]:
y

[12.0, 10]

Si $y$ tiene valores distintos de $0$, se realiza un *update*:

In [62]:
x=[2]*n
y=[-1]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [63]:
y

[11.0, 9]

**Comentarios:**

* En la versión *row oriented* del algoritmo *gaxpy*, el **inner loop** realiza **productos punto** entre el $i$-ésimo renglón de $A$ y el vector $x$. Se realizan $m$ productos punto.

* Obsérvese que el acceso a la matriz $A$ es por renglón.

También puede escribirse al algoritmo *gaxpy* en una forma orientada por columnas:

### Algoritmo gaxpy *column oriented*

Para este algoritmo visualizamos al producto matriz-vector como una combinación lineal de las columnas de $A$:

$$Ax = \displaystyle \sum_{j=1}^n a_jx_j$$

con $a_j$ la $j$-ésima columna de $A$.


In [64]:
x=[2]*n
y=[0]*m
for j in range(n):
    for i in range(m):
        y[i]+=A[i][j]*x[j]

In [65]:
y

[12.0, 10]

**Obs:**

* El algoritmo de multiplicación matriz-vector (versión *row* o *column* oriented) involucra $\mathcal{O}(mn)$ operaciones o una cantidad **cuadrática** de trabajo, que podemos entender como "si duplicamos cada dimensión de $A$ entonces la cantidad de trabajo se incrementa por un factor de $4$". Tal número de operaciones trabajan sobre una matriz o sobre una cantidad **cuadrática** de datos. A los algoritmos que realizan una cantidad cuadrática de trabajo sobre una cantidad cuadrática de datos se les cataloga de **nivel BLAS 2**. Ver [level 2](http://www.netlib.org/blas/#_level_2) para más ejemplos de algoritmos en el álgebra lineal en esta categoría.

* La versión *column oriented* se puede analizar desde el punto de vista puramente algorítmico como un intercambio entre las líneas con los índices $i$ y $j$ de cada *loop* y un acceso a los datos de la matriz por columna. O bien, se puede analizar desde el álgebra lineal indicando que el vector $y$ está en el **espacio generado** por las columnas de $A$ y cuyas coordenadas están siendo dadas por las entradas del vector $x$:

<img src="https://dl.dropboxusercontent.com/s/6a2b7rjs4a71sni/combinacion_lineal_columnas_A.png?dl=0" heigth="700" width="700">

Una ejemplo de visualización del espacio generado por las columnas de $A$, llamado **rango o imagen** de $A$, $Im(A)$, es el siguiente:

<img src="https://dl.dropboxusercontent.com/s/zkbhzv9a2jiw11b/espacio_generado_columnas_de_A.png?dl=0" heigth="400" width="400">

En este dibujo los vectores $b, r(x) \in \mathbb{R}^3$ no están en $Im(A) \subset \mathbb{R}^3$ pero $Ax$ en azul sí.

* Obsérvese que el **inner loop** de la versión *column oriented* en *gaxpy* es un **saxpy** en la que el escalar está dado por una entrada de $x$. Esto lo podemos realizar de forma explícita definiendo $A[:,j]$ a la $j$-ésima columna de $A$ por lo que $A = [A[:,1] | A[:,2] | \dots | A[:,n]]$, entonces:

```
for j=1:n
    y+=A[:,j] * x[j]
 
```

sin embargo como hemos visto, en Python con su implementación más común CPython, no es posible realizar tal indexado:

In [66]:
x=[2]*n
y=[0]*m
for j in range(n):
    y+=A[:,j]*x[j]

TypeError: list indices must be integers or slices, not tuple

a menos que incorporemos alguna paquetería que permita la **vectorización**. Un ejemplo es con [numpy](https://numpy.org/):

In [67]:
import numpy as np

In [68]:
x = 2*np.ones(n)
y = np.zeros(m)

In [69]:
x

array([2., 2., 2., 2., 2.])

In [70]:
y

array([0., 0.])

In [71]:
A

[[1.2, 1.2, 1.2, 1.2, 1.2], [1, 1, 1, 1, 1]]

In [72]:
A=np.array([[1.2,1.2,1.2,1.2,1.2],[1,1,1,1,1]])

In [73]:
A

array([[1.2, 1.2, 1.2, 1.2, 1.2],
       [1. , 1. , 1. , 1. , 1. ]])

In [74]:
for j in range(n):
    y+=A[:,j]*x[j]

In [75]:
y

array([12., 10.])

**Comentarios:**

* La vectorización como se mencionó en [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb) es una herramienta para escribir programas de alto rendimiento pues incrementa el número de instrucciones por ciclo [IPC](https://en.wikipedia.org/wiki/Instructions_per_cycle) para procesadores que caen en la categoría Single Instruction Multiple Data (SIMD) de la taxonomía de Flynn (ver [liga](https://en.wikipedia.org/wiki/Flynn%27s_taxonomy)). Como ejemplo de tales procesadores están los procesadores vectoriales o en arreglo, ver [liga](https://en.wikipedia.org/wiki/Vector_processor).



In [39]:
%%file mult_matrix_vector.py
m=10**3
n=10**2
x=[2]*n
y=[0]*m
A=[[1.2]*n if i%2==0 else [1]*n for i in range(m)]
for j in range(n):
    for i in range(m):
        y[i]+=A[i][j]*x[j]

Overwriting mult_matrix_vector.py


In [40]:
%%file mult_matrix_vector_numpy.py
import numpy as np
m=10**3
n=10**2
x = 2*np.ones(n)
y = np.zeros(m)
#A=np.array([[1.2,1.2,1.2,1.2,1.2],[1,1,1,1,1]])
A=np.random.rand(m,n)
for j in np.arange(n):
    y+=A[:,j]*x[j]

Overwriting mult_matrix_vector_numpy.py


In [41]:
%%bash
sudo perf stat -r 5 python3 mult_matrix_vector.py


 Performance counter stats for 'python3 mult_matrix_vector.py' (5 runs):

         38.996079      task-clock (msec)         #    0.995 CPUs utilized            ( +-  5.47% )
                 0      context-switches          #    0.005 K/sec                    ( +-100.00% )
                 0      cpu-migrations            #    0.000 K/sec                  
              1307      page-faults               #    0.034 M/sec                    ( +-  0.07% )
         147819328      cycles                    #    3.791 GHz                      ( +-  5.54% )
         274120034      instructions              #    1.85  insn per cycle           ( +-  1.28% )
          58988073      branches                  # 1512.667 M/sec                    ( +-  1.40% )
            713721      branch-misses             #    1.21% of all branches          ( +-  2.49% )

       0.039187839 seconds time elapsed                                          ( +-  5.46% )



In [42]:
%%bash
sudo perf stat -r 5 python3 mult_matrix_vector_numpy.py


 Performance counter stats for 'python3 mult_matrix_vector_numpy.py' (5 runs):

        649.818589      task-clock (msec)         #    4.662 CPUs utilized            ( +-  2.32% )
              7115      context-switches          #    0.011 M/sec                    ( +- 99.54% )
                 2      cpu-migrations            #    0.002 K/sec                    ( +- 25.00% )
              4796      page-faults               #    0.007 M/sec                    ( +-  0.15% )
        2472174866      cycles                    #    3.804 GHz                      ( +-  2.31% )
        1284341054      instructions              #    0.52  insn per cycle           ( +-  1.13% )
         255582235      branches                  #  393.313 M/sec                    ( +-  0.86% )
           6505995      branch-misses             #    2.55% of all branches          ( +-  0.38% )

       0.139397445 seconds time elapsed                                          ( +-  2.47% )



In [29]:
%%bash
valgrind --tool=callgrind python3 mult_matrix_vector.py

==4620== Callgrind, a call-graph generating cache profiler
==4620== Copyright (C) 2002-2017, and GNU GPL'd, by Josef Weidendorfer et al.
==4620== Using Valgrind-3.13.0 and LibVEX; rerun with -h for copyright info
==4620== Command: python3 mult_matrix_vector.py
==4620== 
==4620== For interactive control, run 'callgrind_control -h'.
==4620== brk segment overflow in thread #1: can't grow to 0x4a44000
==4620== (see section Limitations in user manual)
==4620== NOTE: further instances of this message will not be shown
==4620== 
==4620== Events    : Ir
==4620== Collected : 2050675713
==4620== 
==4620== I   refs:      2,050,675,713


In [30]:
%%bash
callgrind_annotate --show=Ir --threshold=30 callgrind.out.4620

--------------------------------------------------------------------------------
Profile data file 'callgrind.out.4620' (creator: callgrind-3.13.0)
--------------------------------------------------------------------------------
I1 cache: 
D1 cache: 
LL cache: 
Timerange: Basic block 0 - 441620105
Trigger: Program termination
Profiled target:  python3 mult_matrix_vector.py (PID 4620, part 1)
Events recorded:  Ir
Events shown:     Ir
Event sort order: Ir
Thresholds:       30
Include dirs:     
User annotated:   
Auto-annotation:  off

--------------------------------------------------------------------------------
           Ir 
--------------------------------------------------------------------------------
2,050,675,713  PROGRAM TOTALS

--------------------------------------------------------------------------------
         Ir  file:function
--------------------------------------------------------------------------------
567,052,103  ???:_PyEval_EvalFrameDefault [/usr/bin/python3.6]


In [31]:
%%bash
valgrind --tool=callgrind python3 mult_matrix_vector_numpy.py

==4625== Callgrind, a call-graph generating cache profiler
==4625== Copyright (C) 2002-2017, and GNU GPL'd, by Josef Weidendorfer et al.
==4625== Using Valgrind-3.13.0 and LibVEX; rerun with -h for copyright info
==4625== Command: python3 mult_matrix_vector_numpy.py
==4625== 
==4625== For interactive control, run 'callgrind_control -h'.
==4625== 
==4625== Events    : Ir
==4625== Collected : 532350342
==4625== 
==4625== I   refs:      532,350,342


In [33]:
%%bash
callgrind_annotate --threshold=30 callgrind.out.4625

--------------------------------------------------------------------------------
Profile data file 'callgrind.out.4625' (creator: callgrind-3.13.0)
--------------------------------------------------------------------------------
I1 cache: 
D1 cache: 
LL cache: 
Timerange: Basic block 0 - 111561312
Trigger: Program termination
Profiled target:  python3 mult_matrix_vector_numpy.py (PID 4625, part 1)
Events recorded:  Ir
Events shown:     Ir
Event sort order: Ir
Thresholds:       30
Include dirs:     
User annotated:   
Auto-annotation:  off

--------------------------------------------------------------------------------
         Ir 
--------------------------------------------------------------------------------
532,350,342  PROGRAM TOTALS

--------------------------------------------------------------------------------
        Ir  file:function
--------------------------------------------------------------------------------
47,016,030  /io/numpy/numpy/random/src/mt19937/mt19937.h:__pyx

In [34]:
%%bash
valgrind --tool=cachegrind python3 mult_matrix_vector.py

==4639== Cachegrind, a cache and branch-prediction profiler
==4639== Copyright (C) 2002-2017, and GNU GPL'd, by Nicholas Nethercote et al.
==4639== Using Valgrind-3.13.0 and LibVEX; rerun with -h for copyright info
==4639== Command: python3 mult_matrix_vector.py
==4639== 
--4639-- warning: L3 cache found, using its data for the LL simulation.
==4639== brk segment overflow in thread #1: can't grow to 0x4a44000
==4639== (see section Limitations in user manual)
==4639== NOTE: further instances of this message will not be shown
==4639== 
==4639== I   refs:      2,368,988,768
==4639== I1  misses:          547,625
==4639== LLi misses:           12,262
==4639== I1  miss rate:          0.02%
==4639== LLi miss rate:          0.00%
==4639== 
==4639== D   refs:      1,001,427,161  (735,666,703 rd   + 265,760,458 wr)
==4639== D1  misses:        4,604,561  (  4,286,431 rd   +     318,130 wr)
==4639== LLd misses:          571,355  (    379,688 rd   +     191,667 wr)
==4639== D1  miss rate:          

In [36]:
%%bash
cg_annotate --show=Ir,I1mr --threshold=10 cachegrind.out.4639

--------------------------------------------------------------------------------
I1 cache:         32768 B, 64 B, 8-way associative
D1 cache:         32768 B, 64 B, 8-way associative
LL cache:         8388608 B, 64 B, 16-way associative
Command:          python3 mult_matrix_vector.py
Data file:        cachegrind.out.4639
Events recorded:  Ir I1mr ILmr Dr D1mr DLmr Dw D1mw DLmw
Events shown:     Ir I1mr
Event sort order: Ir I1mr ILmr Dr D1mr DLmr Dw D1mw DLmw
Thresholds:       10 100 100 100 100 100 100 100 100
Include dirs:     
User annotated:   
Auto-annotation:  off

--------------------------------------------------------------------------------
           Ir    I1mr 
--------------------------------------------------------------------------------
2,368,988,768 547,625  PROGRAM TOTALS

--------------------------------------------------------------------------------
           Ir    I1mr  file:function
--------------------------------------------------------------------------------


In [37]:
%%bash
valgrind --tool=cachegrind python3 mult_matrix_vector_numpy.py

==4645== Cachegrind, a cache and branch-prediction profiler
==4645== Copyright (C) 2002-2017, and GNU GPL'd, by Nicholas Nethercote et al.
==4645== Using Valgrind-3.13.0 and LibVEX; rerun with -h for copyright info
==4645== Command: python3 mult_matrix_vector_numpy.py
==4645== 
--4645-- warning: L3 cache found, using its data for the LL simulation.
==4645== 
==4645== I   refs:      532,527,885
==4645== I1  misses:      4,159,424
==4645== LLi misses:         24,353
==4645== I1  miss rate:        0.78%
==4645== LLi miss rate:        0.00%
==4645== 
==4645== D   refs:      214,181,824  (143,001,993 rd   + 71,179,831 wr)
==4645== D1  misses:      7,835,808  (  6,555,784 rd   +  1,280,024 wr)
==4645== LLd misses:        587,284  (    239,028 rd   +    348,256 wr)
==4645== D1  miss rate:         3.7% (        4.6%     +        1.8%  )
==4645== LLd miss rate:         0.3% (        0.2%     +        0.5%  )
==4645== 
==4645== LL refs:        11,995,232  ( 10,715,208 rd   +  1,280,024 wr)
==464

In [38]:
%%bash
cg_annotate --show=Ir,I1mr --threshold=10 cachegrind.out.4645

--------------------------------------------------------------------------------
I1 cache:         32768 B, 64 B, 8-way associative
D1 cache:         32768 B, 64 B, 8-way associative
LL cache:         8388608 B, 64 B, 16-way associative
Command:          python3 mult_matrix_vector_numpy.py
Data file:        cachegrind.out.4645
Events recorded:  Ir I1mr ILmr Dr D1mr DLmr Dw D1mw DLmw
Events shown:     Ir I1mr
Event sort order: Ir I1mr ILmr Dr D1mr DLmr Dw D1mw DLmw
Thresholds:       10 100 100 100 100 100 100 100 100
Include dirs:     
User annotated:   
Auto-annotation:  off

--------------------------------------------------------------------------------
         Ir      I1mr 
--------------------------------------------------------------------------------
532,527,885 4,159,424  PROGRAM TOTALS

--------------------------------------------------------------------------------
         Ir      I1mr  file:function
---------------------------------------------------------------------------

* El algoritmo *gaxpy row oriented* puede escribirse de forma más compacta haciendo uso de la definición de producto punto estándar: $x^Ty$ para dos vectores columna $x$ y $y$. En el caso de una matriz $A$ se tiene:

```
for i=1:m
    y[i]+=A[i,:]^T*x
```

y en Python:

In [76]:
x = 2*np.ones(n)
y = np.zeros(m)
A=np.array([[1.2,1.2,1.2,1.2,1.2],[1,1,1,1,1]])

In [81]:
for i in range(m):
    y[i]+=A[i,:].dot(x)

In [82]:
y

array([12., 10.])

en donde se utilizó la función [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html) de *numpy*.

* 

**Referencias:**

* E. Anderson, Z. Bai, C. Bischof, L. S. Blackford, J. Demmel, J. Dongarra, J. Du Croz,
A. Greenbaum, S. Hammarling, A. Mckenney and D. Sorensen, LAPACK Users Guide, Society for Industrial and Applied Mathematics, Philadelphia, PA, third ed., 1999.

* G. H. Golub, C. F. Van Loan, Matrix Computations, John Hopkins University
Press, 2013.

* [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb)